In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

In [2]:
# Load all the datasets
telemetry = pd.read_csv('C:/Users/user/Downloads/PdM_telemetry.csv')
errors = pd.read_csv('C:/Users/user/Downloads/PdM_errors.csv')
failures = pd.read_csv('C:/Users/user/Downloads/PdM_failures.csv')
machines = pd.read_csv('C:/Users/user/Downloads/PdM_machines.csv')
maintenance = pd.read_csv('C:/Users/user/Downloads/PdM_maint.csv')

# Check their shapes
print(telemetry.shape)
print(errors.shape)
print(failures.shape)
print(machines.shape)
print(maintenance.shape)

(876100, 6)
(3919, 3)
(761, 3)
(100, 3)
(3286, 3)


In [3]:
# Check the structure and missing values

datasets = {
    "Telemetry": telemetry,
    "Errors": errors,
    "Failures": failures,
    "Machines": machines,
    "Maintenance": maintenance
}

# Loop through each dataset to print info and missing values
for name, df in datasets.items():
    print(f"\n{name} Dataset Info:\n{'-'*25}")
    print(df.info())
    print(f"\n{name} Missing Values:\n{'-'*25}")
    print(df.isnull().sum())




Telemetry Dataset Info:
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876100 entries, 0 to 876099
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   datetime   876100 non-null  object 
 1   machineID  876100 non-null  int64  
 2   volt       876100 non-null  float64
 3   rotate     876100 non-null  float64
 4   pressure   876100 non-null  float64
 5   vibration  876100 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 40.1+ MB
None

Telemetry Missing Values:
-------------------------
datetime     0
machineID    0
volt         0
rotate       0
pressure     0
vibration    0
dtype: int64

Errors Dataset Info:
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919 entries, 0 to 3918
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   3919 non-null   object
 1   machineID  

In [4]:
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'])


In [6]:
# Convert datetime columns to datetime dtype
errors['datetime'] = pd.to_datetime(errors['datetime'])
failures['datetime'] = pd.to_datetime(failures['datetime'])
maintenance['datetime'] = pd.to_datetime(maintenance['datetime'])


In [7]:
print(telemetry.duplicated().sum())
print(errors.duplicated().sum())
print(failures.duplicated().sum())
print(machines.duplicated().sum())
print(maintenance.duplicated().sum())


0
0
0
0
0


In [8]:
# 1. Merge telemetry with machines (machineID is the key)
full_df = telemetry.merge(machines, on='machineID', how='left')

# 2. Merge errors on datetime + machineID
full_df = full_df.merge(errors, on=['datetime', 'machineID'], how='left')

# 3. Merge maintenance on datetime + machineID
full_df = full_df.merge(maintenance, on=['datetime', 'machineID'], how='left')

# 4. Merge failures on datetime + machineID
full_df = full_df.merge(failures, on=['datetime', 'machineID'], how='left')

# 5. Inspect the result shape and head
print(f"Merged dataframe shape: {full_df.shape}")
full_df.head()


Merged dataframe shape: (877209, 11)
             datetime  machineID        volt      rotate    pressure  \
0 2015-01-01 06:00:00          1  176.217853  418.504078  113.077935   
1 2015-01-01 07:00:00          1  162.879223  402.747490   95.460525   
2 2015-01-01 08:00:00          1  170.989902  527.349825   75.237905   
3 2015-01-01 09:00:00          1  162.462833  346.149335  109.248561   
4 2015-01-01 10:00:00          1  157.610021  435.376873  111.886648   

   vibration   model  age errorID comp failure  
0  45.087686  model3   18     NaN  NaN     NaN  
1  43.413973  model3   18     NaN  NaN     NaN  
2  34.178847  model3   18     NaN  NaN     NaN  
3  41.122144  model3   18     NaN  NaN     NaN  
4  25.990511  model3   18     NaN  NaN     NaN  


In [9]:
print(full_df[['errorID', 'comp', 'failure']].isna().sum())


errorID    873285
comp       874241
failure    876087
dtype: int64


In [10]:
full_df['comp'] = full_df['comp'].fillna('none')
full_df.head()

<bound method NDFrame.head of                   datetime  machineID        volt      rotate    pressure  \
0      2015-01-01 06:00:00          1  176.217853  418.504078  113.077935   
1      2015-01-01 07:00:00          1  162.879223  402.747490   95.460525   
2      2015-01-01 08:00:00          1  170.989902  527.349825   75.237905   
3      2015-01-01 09:00:00          1  162.462833  346.149335  109.248561   
4      2015-01-01 10:00:00          1  157.610021  435.376873  111.886648   
...                    ...        ...         ...         ...         ...   
877204 2016-01-01 02:00:00        100  179.438162  395.222827  102.290715   
877205 2016-01-01 03:00:00        100  189.617555  446.207972   98.180607   
877206 2016-01-01 04:00:00        100  192.483414  447.816524   94.132837   
877207 2016-01-01 05:00:00        100  165.475310  413.771670  104.081073   
877208 2016-01-01 06:00:00        100  171.336037  496.096870   79.095538   

        vibration   model  age errorID  comp 

In [11]:
full_df['errorID'] = full_df['errorID'].fillna(0)
full_df['failure'] = full_df['failure'].fillna(0)
full_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0


In [13]:
print(full_df.info())
print(full_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877209 entries, 0 to 877208
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   datetime   877209 non-null  datetime64[ns]
 1   machineID  877209 non-null  int64         
 2   volt       877209 non-null  float64       
 3   rotate     877209 non-null  float64       
 4   pressure   877209 non-null  float64       
 5   vibration  877209 non-null  float64       
 6   model      877209 non-null  object        
 7   age        877209 non-null  int64         
 8   errorID    877209 non-null  object        
 9   comp       877209 non-null  object        
 10  failure    877209 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 73.6+ MB
None
datetime     0
machineID    0
volt         0
rotate       0
pressure     0
vibration    0
model        0
age          0
errorID      0
comp         0
failure      0
dtype: int64


In [14]:
for c in ['comp1', 'comp2', 'comp3', 'comp4']:
    full_df[f'{c}_maint'] = (full_df['comp'] == c).astype(int)
full_df.head()


,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure,comp1_maint,comp2_maint,comp3_maint,comp4_maint
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0,0,0,0,0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0,0,0,0,0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0,0,0,0,0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0,0,0,0,0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0,0,0,0,0


In [15]:
comp_dummies = pd.get_dummies(full_df['comp'], prefix='comp')
full_df = pd.concat([full_df, comp_dummies], axis=1)

full_df.head()


,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure,comp1_maint,comp2_maint,comp3_maint,comp4_maint,comp_comp1,comp_comp2,comp_comp3,comp_comp4,comp_none
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0,0,0,0,0,False,False,False,False,True
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0,0,0,0,0,False,False,False,False,True
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0,0,0,0,0,False,False,False,False,True
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0,0,0,0,0,False,False,False,False,True
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0,0,0,0,0,False,False,False,False,True


In [16]:
comp_columns = ['comp_comp1', 'comp_comp2', 'comp_comp3', 'comp_comp4', 'comp_none']
full_df[comp_columns] = full_df[comp_columns].astype(int)
full_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure,comp1_maint,comp2_maint,comp3_maint,comp4_maint,comp_comp1,comp_comp2,comp_comp3,comp_comp4,comp_none
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0,0,0,0,0,0,0,0,0,1
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0,0,0,0,0,0,0,0,0,1
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0,0,0,0,0,0,0,0,0,1
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0,0,0,0,0,0,0,0,0,1
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0,0,0,0,0,0,0,0,0,1


In [17]:
full_df.drop(['comp1_maint', 'comp2_maint', 'comp3_maint', 'comp4_maint'], axis=1, inplace=True, errors='ignore')


In [18]:
full_df.head()


,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure,comp_comp1,comp_comp2,comp_comp3,comp_comp4,comp_none
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0,0,0,0,0,1
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0,0,0,0,0,1
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0,0,0,0,0,1
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0,0,0,0,0,1
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0,0,0,0,0,1


In [19]:
print(full_df.columns)

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'model', 'age', 'errorID', 'comp', 'failure', 'comp_comp1',
       'comp_comp2', 'comp_comp3', 'comp_comp4', 'comp_none'],
      dtype='object')


In [20]:
full_df.isnull().sum()


datetime      0
machineID     0
volt          0
rotate        0
pressure      0
vibration     0
model         0
age           0
errorID       0
comp          0
failure       0
comp_comp1    0
comp_comp2    0
comp_comp3    0
comp_comp4    0
comp_none     0
dtype: int64

In [21]:
full_df.dtypes


datetime      datetime64[ns]
machineID              int64
volt                 float64
rotate               float64
pressure             float64
vibration            float64
model                 object
age                    int64
errorID               object
comp                  object
failure               object
comp_comp1             int32
comp_comp2             int32
comp_comp3             int32
comp_comp4             int32
comp_none              int32
dtype: object

In [23]:
full_df['year'] = full_df['datetime'].dt.year
full_df['month'] = full_df['datetime'].dt.month
full_df['day'] = full_df['datetime'].dt.day
full_df['hour'] = full_df['datetime'].dt.hour

full_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,model,age,errorID,comp,failure,comp_comp1,comp_comp2,comp_comp3,comp_comp4,comp_none,year,day,hour,month
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,none,0,0,0,0,0,1,2015,1,6,1
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,none,0,0,0,0,0,1,2015,1,7,1
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,none,0,0,0,0,0,1,2015,1,8,1
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,none,0,0,0,0,0,1,2015,1,9,1
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,none,0,0,0,0,0,1,2015,1,10,1


In [27]:
full_df.drop('datetime', axis=1, inplace=True)
full_df.head()


KeyError: "['datetime'] not found in axis"

In [29]:
full_df.drop('comp', axis=1, inplace=True)
full_df.head()

,machineID,volt,rotate,pressure,vibration,model,age,errorID,failure,comp_comp1,comp_comp2,comp_comp3,comp_comp4,comp_none,year,day,hour,month
0,1,176.217853,418.504078,113.077935,45.087686,model3,18,0,0,0,0,0,0,1,2015,1,6,1
1,1,162.879223,402.747490,95.460525,43.413973,model3,18,0,0,0,0,0,0,1,2015,1,7,1
2,1,170.989902,527.349825,75.237905,34.178847,model3,18,0,0,0,0,0,0,1,2015,1,8,1
3,1,162.462833,346.149335,109.248561,41.122144,model3,18,0,0,0,0,0,0,1,2015,1,9,1
4,1,157.610021,435.376873,111.886648,25.990511,model3,18,0,0,0,0,0,0,1,2015,1,10,1


In [34]:
full_df.to_csv("Machine_Processed_data.csv", index=False)
